# Contents:
Set Up

Group By: .groupby(), .agg()

Loyalty Flag: column for max orders per customer, loyalty flag

Spending Flag: column for mean spending, spending amount flag

Frequency Flag: column for median days btw purchases, frequency flag

## Set Up

In [1]:
#set up
import pandas as pd
import numpy as np
import os
path = r"C:\Users\irkat\OneDrive - University of North Carolina at Charlotte\Desktop\Data Cert\A4"

In [2]:
#import
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared', 'orders_products_merged.pkl'))

In [3]:
#test
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busy_day,busiest_days,busy_time,busy_times
0,2539329,1,1,2,8,11.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Fewest Orders,Fewest Orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy,Most Orders,Most Orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy,Most Orders,Most Orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Least busy,Most Orders,Most Orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Least busy,Most Orders,Most Orders


In [4]:
df.shape

(32404859, 18)

## Group By

In [5]:
#avg order number by department
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


There are results for every department. For the departments in both, the means are slightly lower then the subset.

## Loyalty flag

In [6]:
#create column for max number of orders by user
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
#flagging loyal
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
#flagging regular
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
#flagging new
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
#compare prices of loyalty groups
df.groupby('loyalty_flag').agg({'prices': ['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

Loyal customers tend to spend less then others. New customers spend the most

## Spending flag

In [11]:
#column based on user spending
df['user_spending_mean'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [12]:
df.loc[df['user_spending_mean'] < 10, 'spending_flag'] = 'Low spender'

In [13]:
df.loc[df['user_spending_mean'] >=10, 'spending_flag'] = 'High spender'

In [14]:
#compare prices of spending groups
df.groupby('spending_flag').agg({'prices': ['mean','min','max']})

prices              
                     mean  min      max
spending_flag                          
High spender   222.343698  1.0  99999.0
Low spender      7.780685  1.0     25.0

## Frequency Flag

In [15]:
#column for user's median days between orders
df['days_median'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [16]:
df.loc[df['days_median'] > 20, 'order_frequency_flag'] = 'Non-frequent Customer'

In [17]:
df.loc[(df['days_median'] > 10) & (df['days_median'] <=20) , 'order_frequency_flag'] = 'Regular Customer'

In [18]:
df.loc[df['days_median'] <= 20, 'order_frequency_flag'] = 'Frequent Customer'

In [19]:
#compare prices of frequency groups
df.groupby('order_frequency_flag').agg({'prices': ['mean','min','max']})

prices              
                            mean  min      max
order_frequency_flag                          
Fequent Customer       11.971551  1.0  99999.0
Non-frequent Customer  12.075647  1.0  99999.0

In [20]:
df.to_pickle(os.path.join(path, '02 Data','Prepared', 'orders_products_merged.pkl'))